<a href="https://colab.research.google.com/github/abrham17/SVM_VS_SOFTMAX/blob/main/CNN_VS_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **importing libraries**


*   pytorch for CNN base model creating , optimization algorithms , datasets and dataloders
*   sklearn for support vector component , and acccuracy measure




SVC is a sickit learn implementatio of svm it supports:-


*   C parameter
*   kernel options(linear , rbf , poly)
*   multi class classification via internal one to one classification

for k classes it trains K(K-1)/2 classifiers





In [ ]:
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch import nn
import torch
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import time

# CNN model MNIST classification




In [ ]:
class SimpleCNN(nn.Module):
    """
    A simple Convolutional Neural Network (CNN) architecture for MNIST digit classification.

    This network is designed for grayscale handwritten digit images (28x28) and outputs
    class logits for a given number of categories (default: 10 for MNIST digits 0–9).
    It can also be used as a generic feature extractor by using the `extract_features` method.

    Architecture:
        Input: (batch_size, 1, 28, 28)
        Layer 1:
            - Conv2d(1 → 32 filters, kernel_size=3, padding=1)
            - ReLU activation
            - MaxPool2d(kernel_size=2, stride=2) → output shape: (batch_size, 32, 14, 14)
        Layer 2:
            - Conv2d(32 → 64 filters, kernel_size=3, padding=1)
            - ReLU activation
            - MaxPool2d(kernel_size=2, stride=2) → output shape: (batch_size, 64, 7, 7)
        Fully Connected Layers:
            - Flatten → shape: (batch_size, 64*7*7)
            - Linear(64*7*7 → 128)
            - ReLU activation
            - Linear(128 → num_classes) → produces raw logits

    Attributes:
        conv1 (nn.Conv2d): First convolutional layer mapping 1 input channel to 32 feature maps.
        conv2 (nn.Conv2d): Second convolutional layer mapping 32 feature maps to 64 feature maps.
        pool (nn.MaxPool2d): Max pooling operation with kernel size 2 and stride 2.
        fc1 (nn.Linear): First fully connected layer mapping flattened features to 128 hidden units.
        fc2 (nn.Linear): Output fully connected layer mapping 128 hidden units to `num_classes` logits.

    Args:
        num_classes (int): Number of output classes. Defaults to 10.

    Methods:
        forward(x):
            Performs the full forward pass of the CNN from input images to final logits.
            Input: Tensor of shape (batch_size, 1, 28, 28)
            Output: Tensor of shape (batch_size, num_classes) containing raw logits.

        extract_features(x):
            Performs a partial forward pass to extract deep features before the final classification layer.
            This is useful for using CNN as a feature extractor for other models (e.g., SVM, kNN).
            Input: Tensor of shape (batch_size, 1, 28, 28)
            Output: Tensor of shape (batch_size, 128) containing high-level features.
    """
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # logits
        return x

    def extract_features(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return x


# data initalization


**Data Preparation for MNIST**
- Convert images from NumPy arrays to PyTorch tensors.
- Normalize pixel values with mean 0.1307 and std 0.3081.
- Load MNIST training and test datasets from torchvision.
- Apply transformations to both sets.
- Create DataLoaders with batch size 64 (shuffle train set).

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)


# training loop
- device intialization if cuda is available use it else use cpu
- used Adam optimizer with lr of 0.01
- loss is cross entrophy loss becuase we a working on multi class classification so cross entrophy applies softmax before working on the loss
- trianing loop with 3 epochs

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

epochs = 3
start = time.time()
for epoch in range(epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")
end = time.time()
print(f"training time {end-start}")

Epoch [1/3], Loss: 0.1134
Epoch [2/3], Loss: 0.2229
Epoch [3/3], Loss: 0.0492
training time 294.05833625793457


# evaluation of CNN model with softmax


In [ ]:
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()

print(f"CNN + Softmax Accuracy: {100 * correct / total:.2f}%")


CNN + Softmax Accuracy: 98.95%


## feature extraction for SVM

- Feature extraction using CNN's extract features method then vertical stacking the train_features and test features

In [ ]:
model.eval()

# Extract train features
train_features, train_labels = [], []
with torch.no_grad():
    for images, labels in train_loader:
        images = images.to(device)
        feats = model.extract_features(images)
        train_features.append(feats.cpu().numpy())
        train_labels.append(labels.numpy())

train_features = np.vstack(train_features)
train_labels = np.hstack(train_labels)

# Extract test features
test_features, test_labels = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        feats = model.extract_features(images)
        test_features.append(feats.cpu().numpy())
        test_labels.append(labels.numpy())

test_features = np.vstack(test_features)
test_labels = np.hstack(test_labels)


# Using SVM from sklearn's SVM.SVC
- initalizing the SVC model from sklearn using linear kernel if needed the kernel can be changed to rbf and poly
- training the SVC with the extracted train features
- at last predicting and measuring the SVC model

In [ ]:
start = time.time()
svm_clf = SVC(kernel='linear')
descion = svm_clf.fit(train_features, train_labels)
end = time.time()
print(f"training time {end-start}")
svm_preds = svm_clf.predict(test_features)
svm_acc = accuracy_score(test_labels, svm_preds)

print(f"CNN (feature extractor) + SVM Accuracy: {svm_acc*100:.2f}%")


training time 7.123582601547241
CNN (feature extractor) + SVM Accuracy: 99.03%


## Get cnn classifier output

Obtain the output of the CNN before the final softmax layer.


In [ ]:
model.eval()

n_samples = 5000
subset_indices = np.arange(n_samples)
subset_images = []
subset_labels = []

# Assuming train_dataset is a standard PyTorch Dataset
for i in subset_indices:
    image, label = train_dataset[i]
    subset_images.append(image)
    subset_labels.append(label)

# Stack images and convert to tensor
subset_images_tensor = torch.stack(subset_images).to(device)
subset_labels_tensor = torch.tensor(subset_labels).to(device)


# Get CNN outputs before softmax
with torch.no_grad():
    cnn_outputs_subset = model(subset_images_tensor).cpu().numpy()


## Apply dimensionality reduction to cnn output



In [ ]:
tsne_cnn = TSNE(n_components=2, random_state=42, n_jobs=-1)
cnn_outputs_2d = tsne_cnn.fit_transform(cnn_outputs_subset)

NameError: name 'TSNE' is not defined

## Visualize the cnn classifier output



In [ ]:
plt.figure(figsize=(10, 8))
scatter = plt.scatter(cnn_outputs_2d[:, 0], cnn_outputs_2d[:, 1], c=subset_labels, cmap='tab10', s=10)
plt.title("t-SNE Visualization of CNN Outputs")
plt.colorbar(scatter, label="Digit Label")
plt.show()

## Get svm classifier decision function output


In [ ]:
svm_decision_subset = svm_clf.decision_function(train_features_subset)

## Apply dimensionality reduction to svm output



In [ ]:
tsne_svm_decision = TSNE(n_components=2, random_state=42, n_jobs=-1)
svm_decision_2d = tsne_svm_decision.fit_transform(svm_decision_subset)

## Visualize the svm classifier output


In [ ]:
plt.figure(figsize=(10, 8))
scatter = plt.scatter(svm_decision_2d[:, 0], svm_decision_2d[:, 1], c=subset_labels, cmap='tab10', s=10)
plt.title("t-SNE Visualization of SVM Decision Function Outputs")
plt.colorbar(scatter, label="Digit Label")
plt.show()

## Visualize svm decision boundary


In [ ]:
# Train a new SVM on the 2D t-SNE features
svm_clf_2d = SVC(kernel='linear')
svm_clf_2d.fit(train_features_2d, train_labels_subset)

# Create a meshgrid
x_min, x_max = train_features_2d[:, 0].min() - 1, train_features_2d[:, 0].max() + 1
y_min, y_max = train_features_2d[:, 1].min() - 1, train_features_2d[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                     np.arange(y_min, y_max, 0.1))

# Predict decision function values for the meshgrid
Z = svm_clf_2d.decision_function(np.c_[xx.ravel(), yy.ravel()])

# Reshape the decision function output for contour plotting
# For multi-class with decision_function, Z will have shape (n_samples, n_classes)
# We need to choose how to represent the boundary. A common way is to show boundaries
# between all pairs of classes, but for a single contour plot, we can visualize
# the boundary of one-vs-rest or the predicted class.
# Let's visualize the predicted class for simplicity in the contour plot.
Z = svm_clf_2d.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)


# Plot the contour of the decision boundary
plt.figure(figsize=(10, 8))
plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.8)

# Overlay the scatter plot of the data points
scatter = plt.scatter(train_features_2d[:, 0], train_features_2d[:, 1], c=train_labels_subset, cmap='tab10', s=10, edgecolors='k')

# Add title and labels
plt.title("SVM Decision Boundary on t-SNE Reduced Features")
plt.xlabel("t-SNE Component 1")
plt.ylabel("t-SNE Component 2")
plt.colorbar(scatter, label="Digit Label")
plt.show()